# LCEL 핵심 개념

## 파이프라인(Pipeline)

**LCEL의 핵심은 파이프라인 연산자 `|`** 입니다. 이는 **Unix 파이프라인**에서 영감을 받은 개념으로, 하나의 출력이 다음 단계의 입력으로 자연스럽게 전달됩니다.

#### 파이프라인 흐름 예시

LCEL 은 동일한 방식으로 동작한다:

```python
# LCEL 체인 구성
chain = prompt_template | model | output_parser
#            ↑              ↑          ↑
# 1단계: 프롬프트 생성 → 2단계: 모델 추론 → 3단계: 결과 파싱
```

### 기본 체인의 3단계

#### PromptTemplate - 지시사항 준비
- **역할**: 사용자 입력을 AI가 이해할 수 있는 형태로 변환
- **입력**: 딕셔너리 형태의 변수들 (`{"topic": "인공지능"}`)
- **출력**: 완성된 프롬프트 문자열

#### Model - AI 처리
- **역할**: 프롬프트를 받아 AI가 답변 생성
- **입력**: 포맷팅된 프롬프트 텍스트
- **출력**: AIMessage 객체 (내용 + 메타데이터)

#### OutputParser - 결과 정리
- **역할**: AI 응답을 사용하기 쉬운 형태로 변환  
- **입력**: AIMessage 객체
- **출력**: 순수 텍스트 또는 구조화된 데이터

LCEL 은 전체 과정을 한 번의 호출로 실행할 수 있도록 단순화한다.

---

## PromptTemplate 개요

**PromptTemplate** 은 **동적 프롬프트** 를 만들어주는 강력한 도구입니다. 마치 **편지 양식** 에 이름만 바꿔서 여러 사람에게 보내는 것과 같습니다.

In [6]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

True

In [7]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-Tutorial


### PromptTemplate 의 구성 요소

**PromptTemplate** 은 **템플릿 엔진** 의 역할을 하며, 사용자 입력을 받아 **동적으로 프롬프트를 생성** 합니다.

#### 핵심 구성 요소

1. template: 실제 프롬프트 내용이 담긴 문자열 템플릿
   ```python
   template = "{country}의 수도는 어디인가요?"
   ```

2. input_variables: 템플릿에서 사용할 변수들의 이름 목록
   ```python
   # 중괄호 {} 안의 변수명들이 input_variables가 됨
   # 위 예시에서는 ["country"]가 자동으로 추출됨
   ```

#### 템플릿 변수 사용법

- **중괄호 `{}`** 안에 변수명 작성
- **여러 변수** 사용 가능: `"{name}님, {city}의 날씨는 어떤가요?"`
- **변수명은 영문자로 시작** , 숫자와 언더스코어 사용 가능

#### from_template 메서드

```python
# 간단한 방법 - from_template() 사용
prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

# 직접 생성
prompt = PromptTemplate(
    template="{country}의 수도는 어디인가요?",
    input_variables=["country"]
)
```

from_template 의 장점

- 자동 변수 추출: 중괄호 안의 변수를 자동 인식
- 간결한 코드: 한 줄로 템플릿 생성
- 오류 감소: 변수명 오타/누락 가능성 축소


In [46]:
# 스트리밍 출력을 위한 헬퍼 함수와 PromptTemplate 클래스 임포트
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate

### PromptTemplate 객체 생성

`from_template()` 메서드를 사용하면 간단하게 PromptTemplate 객체를 생성할 수 있다.

In [8]:
# 동적 프롬프트 템플릿 정의 - {country} 부분이 변수로 대체됨
template = "{country}의 수도는 어디인가요?"

# from_template 메서드를 이용하여 PromptTemplate 객체 생성
# 중괄호 안의 변수들이 자동으로 input_variables로 인식됨
prompt_template = PromptTemplate.from_template(template)

# 생성된 PromptTemplate 객체 확인
prompt_template

NameError: name 'PromptTemplate' is not defined

In [5]:
# 템플릿에 구체적인 값을 대입하여 완성된 프롬프트 생성
prompt = prompt_template.format(country="대한민국")
print(f"생성된 프롬프트: {prompt}")

생성된 프롬프트: 대한민국의 수도는 어디인가요?


In [6]:
# 다른 국가로 변경하여 프롬프트 생성 테스트
prompt = prompt_template.format(country="미국")
print(f"생성된 프롬프트: {prompt}")

생성된 프롬프트: 미국의 수도는 어디인가요?


In [7]:
# ChatOpenAI 모델 임포트 및 초기화
from langchain_openai import ChatOpenAI
import os

# OpenRouter 기반 모델 객체 생성
model = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

In [8]:
response = model.invoke(prompt)
print(response.content)

미국의 수도는 워싱턴 D.C.(Washington, D.C.)입니다.


---

## LCEL 체인 생성

### LCEL(LangChain Expression Language) 심화

![LCEL Pipeline](./images/lcel.png)

**LCEL의 핵심은 `|` (파이프) 연산자** 입니다. 이를 통해 여러 구성 요소를 **체인처럼 연결** 하여 하나의 통합된 워크플로우를 만들 수 있습니다.

### 파이프라인 연산자의 작동 원리

```python
chain = prompt_template | model | output_parser
```

#### 데이터 흐름 과정

1. 입력: `{"topic": "인공지능"}` (딕셔너리)
2. 1단계: `prompt_template` → 완성된 프롬프트 텍스트
3. 2단계: `model` → AIMessage 객체 (AI 응답)  
4. 3단계: `output_parser` → 최종 텍스트 결과

```python
# LCEL 파이프라인 예시  
chain = prompt | model | parser
```

공통점

- 왼쪽에서 오른쪽으로 데이터 흐름
- 각 단계의 출력이 다음 단계의 입력이 됨
- 모듈화: 단계별 교체/재사용 가능

### LCEL 장점

자동 최적화

- 병렬 처리: 가능한 부분은 동시에 실행
- 메모리 효율성: 중간 결과의 효율적 관리
- 스트리밍: 실시간 결과 출력 지원

개발자 친화적

- 문법이 직관적이며 데이터 흐름을 파악하기 용이
- 단계별 결과 추적이 쉬움
- 구성요소 재사용이 용이

간단한 체인을 구성한다.

In [16]:
# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요.")

# ChatOpenAI 모델 객체 생성
import os

model = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 기본 체인 구성 (출력 파서 없이) - 프롬프트와 모델만 연결
chain = prompt | model

### invoke 메서드

invoke 는 LCEL 체인을 실행하는 기본 메서드이다.

사용법
- **입력 형태**: Python 딕셔너리 `{"변수명": "값"}`
- **실행 방식**: 동기식 (결과가 나올 때까지 대기)
- **반환값**: 체인의 최종 출력 (출력 파서에 따라 달라짐)

In [19]:
# 체인 실행을 위한 입력 딕셔너리 정의
# 키는 템플릿의 변수명과 일치해야 함
input = {"topic": "인공지능 모델의 학습 원리"}

In [20]:
# LCEL 체인 실행: 프롬프트 생성 → 모델 처리 → AIMessage 반환
# invoke() 메서드로 전체 파이프라인을 한 번에 실행
result = chain.invoke(input)

# 결과 출력 (AIMessage 객체 형태로 반환됨)
result

AIMessage(content='네, 인공지능(AI) 모델의 학습 원리를 쉽게 설명해드릴게요.\n\n---\n\n### 1. **예시로 이해하기**\nAI 모델을 사람에 비유해볼게요. 예를 들어, 아이가 사과와 오렌지를 구분하는 법을 배운다고 생각해봅시다.\n\n- **데이터 제공:** 아이에게 사과와 오렌지 사진을 보여주면서 "이건 사과야", "이건 오렌지야"라고 알려줍니다.\n- **패턴 찾기:** 아이는 여러 사진을 보면서 "사과는 보통 빨갛고, 오렌지는 주황색이구나"처럼 특징을 스스로 찾아냅니다.\n- **새로운 문제:** 이제 아이에게 처음 보는 과일 사진을 보여주면, 배운 특징을 바탕으로 "이건 사과야!"라고 맞힐 수 있게 됩니다.\n\n---\n\n### 2. **AI 모델의 학습 과정**\nAI도 비슷하게 학습합니다.\n\n1. **데이터 수집:** AI에게 많은 예시(데이터)를 줍니다. 예를 들어, 고양이와 강아지 사진과 각각의 정답(라벨)을 줍니다.\n2. **모델 설계:** AI 모델(뇌 역할을 하는 프로그램)을 만듭니다.\n3. **학습(Training):** AI는 데이터를 보면서 "고양이는 귀가 뾰족하고, 강아지는 귀가 처졌네"처럼 스스로 규칙(패턴)을 찾아냅니다.\n4. **예측:** 학습이 끝난 후, 처음 보는 사진을 보여주면 AI가 "이건 고양이야"라고 예측합니다.\n5. **정답과 비교:** 예측이 맞았는지 틀렸는지 확인하고, 틀렸다면 AI가 스스로 규칙을 조금씩 수정합니다.\n6. **반복:** 이 과정을 수천~수백만 번 반복하면서 점점 더 똑똑해집니다.\n\n---\n\n### 3. **핵심 원리**\n- **데이터에서 패턴을 찾는다.**\n- **정답과 비교하며 스스로 규칙을 수정한다.**\n- **반복해서 점점 더 정확해진다.**\n\n---\n\n### 4. **쉽게 요약**\n> **AI 모델의 학습은 많은 예시를 보면서 스스로 규칙을 찾아내고, 틀릴 때마다 규칙을 고쳐가며 점점 더 똑똑해지는 과정입니다.**\n\n

### 스트리밍 출력

스트리밍 은 모델이 생성한 토큰을 순차적으로 전송하여 실시간으로 응답을 확인하는 방식이다. 긴 응답에서도 대기 시간을 단축한다.

In [21]:
# 스트리밍 방식으로 체인 실행 - 실시간으로 응답 생성 과정 확인
answer = chain.stream(input)

# langchain_teddynote의 헬퍼 함수로 스트리밍 출력을 깔끔하게 표시
stream_response(answer)

네, 인공지능 모델의 학습 원리를 쉽게 설명해드릴게요!

---

### 1. **예시로 이해하기**
인공지능 모델이 학습하는 과정을 사람에게 비유해볼 수 있어요.

- **예시:** 고양이와 강아지 사진을 구분하는 법을 배우는 상황을 생각해봅시다.

### 2. **데이터를 보여주기**
먼저, 많은 고양이 사진과 강아지 사진을 보여줍니다. 각 사진에는 "고양이" 또는 "강아지"라는 정답(라벨)이 붙어 있습니다.

### 3. **패턴 찾기**
모델은 사진을 보면서 고양이와 강아지의 특징(예: 귀 모양, 얼굴 형태 등)을 스스로 찾아내려고 합니다.

### 4. **예측하기**
새로운 사진이 들어오면, 모델은 지금까지 배운 특징을 바탕으로 "이건 고양이야!" 또는 "이건 강아지야!"라고 예측합니다.

### 5. **틀리면 고치기**
처음에는 많이 틀릴 수 있습니다. 틀린 답을 내면, 정답과 비교해서 "어디서 잘못됐는지"를 확인하고, 모델 내부의 계산 방법(가중치 등)을 조금씩 수정합니다.

### 6. **반복해서 연습하기**
이 과정을 수천, 수만 번 반복하면서 점점 더 정확하게 구분할 수 있게 됩니다.

---

### **정리**
- **많은 예시(데이터)를 보고**
- **특징(패턴)을 스스로 찾고**
- **예측을 해보고**
- **틀린 부분을 고치면서**
- **점점 더 똑똑해지는 것**이 인공지능 모델의 학습 원리입니다!

---

궁금한 점이 있으면 더 물어보세요! 😊

---

## OutputParser - 출력 후처리

**OutputParser** 는 AI 의 복잡한 응답을 **사용하기 쉬운 형태로 변환** 해주는 마지막 단계입니다.

In [22]:
# 문자열 출력 파서 임포트
from langchain_core.output_parsers import StrOutputParser

# StrOutputParser 객체 생성 - AIMessage에서 순수 텍스트만 추출
output_parser = StrOutputParser()

### 완전한 체인 구성

이제 **3단계 파이프라인** 을 완성해봅시다: **PromptTemplate → Model → OutputParser**

In [23]:
# 완전한 LCEL 체인 구성: 프롬프트 → 모델 → 출력 파서
# 이제 결과가 AIMessage가 아닌 순수 문자열로 반환됨
chain = prompt | model | output_parser

In [24]:
# 완성된 체인으로 invoke 실행 - 이제 순수 문자열이 반환됨
input = {"topic": "인공지능 모델의 학습 원리"}
result = chain.invoke(input)

# 결과 출력 (이제 문자열 형태로 깔끔하게 출력됨)
print("=== 완성된 체인 결과 ===")
print(result)

=== 완성된 체인 결과 ===
네, 인공지능 모델의 학습 원리를 쉽게 설명해드릴게요!

---

### 1. **예시로 배우기**
인공지능 모델은 사람처럼 예시(데이터)를 보면서 배우는 프로그램입니다. 예를 들어, 고양이와 강아지 사진을 구분하는 모델을 만든다고 해볼게요.

---

### 2. **입력과 정답**
- **입력:** 고양이 사진, 강아지 사진 등
- **정답:** 이 사진은 고양이, 저 사진은 강아지

이렇게 입력(사진)과 정답(고양이/강아지)을 쌍으로 많이 보여줍니다.

---

### 3. **예측과 비교**
모델은 사진을 보고 "이건 고양이야!"라고 예측합니다. 그리고 실제 정답과 비교해서 맞았는지 틀렸는지 확인합니다.

---

### 4. **틀린 만큼 고치기**
만약 틀렸다면, "왜 틀렸지?"를 계산해서 모델 내부의 규칙(수학적인 값, 즉 '가중치')을 조금씩 고칩니다. 이 과정을 반복하면서 점점 더 잘 맞추게 됩니다.

---

### 5. **반복 학습**
수많은 사진을 보면서 예측→비교→수정 과정을 반복합니다. 그러면 모델은 점점 더 똑똑해져서, 처음 보는 사진도 잘 맞출 수 있게 됩니다.

---

### **정리**
- 인공지능 모델은 예시(데이터)를 많이 보고,
- 예측을 해보고,
- 정답과 비교해서,
- 틀린 만큼 내부 규칙을 고치면서,
- 점점 더 잘 맞추게 되는 원리입니다.

사람이 문제를 풀고 틀린 부분을 공부해서 실력이 느는 것과 비슷하다고 생각하면 됩니다!


In [25]:
# 완성된 체인으로 스트리밍 실행
answer = chain.stream(input)

print("=== 스트리밍 출력 ===")
# 실시간으로 문자열이 생성되는 과정을 관찰
stream_response(answer)

=== 스트리밍 출력 ===
네, 인공지능 모델의 학습 원리를 쉽게 설명해드릴게요.

---

### 1. **예시로 이해하기**
인공지능 모델을 "학생"이라고 생각해보세요. 이 학생에게 "고양이 사진"과 "강아지 사진"을 많이 보여주면서, 각각이 무엇인지 알려줍니다. 이 과정을 "학습"이라고 해요.

---

### 2. **학습의 과정**
1. **데이터 준비**  
   - 학생에게 보여줄 사진(데이터)을 많이 모아요.
   - 각 사진이 고양이인지, 강아지인지 정답(라벨)도 함께 준비해요.

2. **패턴 찾기**  
   - 학생(모델)은 사진을 보면서 고양이와 강아지의 차이점을 스스로 찾아내려고 해요.
   - 예를 들어, 귀 모양, 얼굴 형태, 꼬리 등에서 차이를 발견할 수 있어요.

3. **예측하기**  
   - 이제 새로운 사진을 보여주면, 학생이 "이건 고양이야!" 또는 "이건 강아지야!"라고 맞혀보려고 해요.

4. **틀린 답 수정하기**  
   - 만약 학생이 틀리면, "이건 고양이가 아니라 강아지야"라고 알려줘요.
   - 학생은 왜 틀렸는지 생각하고, 다음에는 더 잘 맞히려고 노력해요.
   - 이 과정을 반복하면서 점점 더 똑똑해져요.

---

### 3. **수학적으로는?**
- 인공지능 모델은 수많은 숫자(파라미터)를 조정하면서, 정답에 더 가까워지도록 "학습"해요.
- 이 과정을 "최적화"라고 부르고, 컴퓨터가 자동으로 계산해줘요.

---

### 4. **정리**
- **많은 예시(데이터)를 보여주고**
- **틀린 부분을 고치면서**
- **점점 더 잘 맞히게 되는 것**  
이게 바로 인공지능 모델의 학습 원리입니다!

---

궁금한 점이 있으면 더 물어보세요!

---

## 실습: 영어 회화 튜터 예제

이제 배운 내용을 활용해서 **실용적인 영어 학습 도우미** 를 만들어봅니다. 

### 프로젝트 개요

- **목표**: 상황별 영어 회화 생성 + 한글 번역 제공
- **특징**: 체계적인 포맷으로 학습 효과 극대화
- **활용**: 다양한 상황에 맞는 영어 표현 학습

In [34]:
# 전문적인 영어 회화 튜터 프롬프트 템플릿 설계
template = """You are an experienced English conversation teacher with 10 years of expertise.
Create practical English conversations for the given situation with Korean translations.
Please follow the FORMAT exactly as shown below.

#SITUATION:
{question}

#FORMAT:
- English Conversation:
- Korean Translation:
- Useful Expressions:
- Cultural Notes (if applicable, provide relevant cultural insights in KOREAN):
"""

# 개선된 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 객체 생성 (OpenRouter 사용)
import os

model = ChatOpenAI(
    temperature=0.5,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 문자열 출력 파서 생성
output_parser = StrOutputParser()

In [35]:
# 영어 회화 튜터 체인 구성
# 프롬프트 → 모델 → 출력 파서의 완전한 파이프라인
chain = prompt | model | output_parser

In [33]:
# 첫 번째 상황: 식당에서 음식 주문하기
situation_1 = "저는 식당에 가서 음식을 주문하고 싶어요"

print("🍽️ === 식당 주문 상황 ===")
print(chain.invoke({"question": situation_1}))

🍽️ === 식당 주문 상황 ===
#SITUATION:  
저는 식당에 가서 음식을 주문하고 싶어요

- English Conversation:  
Waiter: Hello! Welcome to our restaurant. How many people are in your party?  
Customer: Hi! Just one, please.  
Waiter: Right this way. Here is the menu.  
Customer: Thank you.  
Waiter: Are you ready to order?  
Customer: Yes, I’d like the chicken pasta, please.  
Waiter: Would you like anything to drink?  
Customer: Just water, please.  
Waiter: Sure. Your order will be ready soon.

- Korean Translation:  
웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?  
손님: 안녕하세요! 저 혼자예요.  
웨이터: 이쪽으로 오세요. 메뉴판입니다.  
손님: 감사합니다.  
웨이터: 주문하시겠어요?  
손님: 네, 치킨 파스타로 주세요.  
웨이터: 음료는 무엇으로 드릴까요?  
손님: 물로 주세요.  
웨이터: 알겠습니다. 곧 준비해드릴게요.

- Useful Expressions:  
- I’d like to order, please. (주문하고 싶어요.)  
- Can I see the menu? (메뉴판 볼 수 있을까요?)  
- What do you recommend? (추천 메뉴가 뭐예요?)  
- I’ll have the _____. (_____로 할게요.)  
- Could I get this without onions? (양파 빼고 해주실 수 있나요?)  
- Can I get the bill, please? (계산서 주세요.)

- Cultural Notes (i

In [36]:
# 두 번째 상황: 스트리밍으로 실시간 학습 경험
situation_2 = "미국에서 피자 주문"

print("🍕 === 미국 피자 주문 상황 (스트리밍) ===")
# 스트리밍으로 영어 회화가 실시간으로 생성되는 과정 관찰
answer = chain.stream({"question": situation_2})
stream_response(answer)

🍕 === 미국 피자 주문 상황 (스트리밍) ===
#SITUATION:  
미국에서 피자 주문

- English Conversation:  
A: Hi, I’d like to order a pizza for delivery, please.  
B: Sure! What size would you like—small, medium, or large?  
A: I’ll take a large, please.  
B: Great. What toppings would you like?  
A: Pepperoni and mushrooms, please.  
B: Got it. Would you like anything to drink or any sides?  
A: Yes, I’ll have a bottle of cola and some garlic bread.  
B: Perfect. Can I have your address and phone number, please?  
A: Sure, it’s 123 Maple Street, and my number is 555-1234.  
B: Thank you! Your total is $22.50. Your order will arrive in about 40 minutes.  
A: Thank you very much!  
B: You’re welcome. Enjoy your meal!

- Korean Translation:  
A: 안녕하세요, 피자 배달 주문하려고요.  
B: 네! 어떤 사이즈로 드릴까요? 스몰, 미디엄, 라지 중에서요.  
A: 라지로 할게요.  
B: 알겠습니다. 토핑은 무엇으로 하시겠어요?  
A: 페퍼로니랑 버섯으로 해주세요.  
B: 네, 확인했습니다. 음료나 사이드 메뉴도 필요하신가요?  
A: 네, 콜라 한 병이랑 갈릭 브레드도 주세요.  
B: 좋아요. 주소와 전화번호 알려주시겠어요?  
A: 네, 123 메이플 스트리트이고, 전화번호는 555-1234예요.  
B: 감사합니다!

In [37]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

#SITUATION:  
저는 식당에 가서 음식을 주문하고 싶어요

- English Conversation:  
Waiter: Hello! Welcome to our restaurant. How many people are in your party?  
You: Hi! Just one, please.  
Waiter: Right this way. Here is the menu.  
You: Thank you.  
Waiter: Are you ready to order?  
You: Yes, I’d like the grilled chicken, please.  
Waiter: Would you like anything to drink?  
You: Just water, please.  
Waiter: Sure. Your order will be ready soon.  
You: Thank you.

- Korean Translation:  
웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?  
당신: 안녕하세요! 저 혼자예요.  
웨이터: 이쪽으로 오세요. 메뉴판입니다.  
당신: 감사합니다.  
웨이터: 주문하시겠어요?  
당신: 네, 구운 치킨 주세요.  
웨이터: 음료는 무엇으로 드릴까요?  
당신: 물만 주세요.  
웨이터: 알겠습니다. 곧 준비해드릴게요.  
당신: 감사합니다.

- Useful Expressions:  
- How many people? (몇 분이세요?)  
- Just one, please. (저 혼자예요.)  
- Are you ready to order? (주문하시겠어요?)  
- I’d like [menu item], please. ([메뉴 이름] 주세요.)  
- Would you like anything to drink? (음료는 무엇으로 드릴까요?)  
- Just water, please. (물만 주세요.)  
- Thank you. (감사합니다.)

- Cultural Notes (if appli

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

# SITUATION:  
미국에서 피자 주문

- English Conversation:  
A: Hi, I’d like to order a large pepperoni pizza for delivery, please.  
B: Sure! Can I get your address and phone number?  
A: My address is 123 Maple Street, and my number is 555-1234.  
B: Great. Would you like any extra toppings or drinks with that?  
A: Yes, please add mushrooms. And I’ll take a bottle of soda.  
B: Got it. Your total is $18.50. It’ll be about 30 minutes.  
A: Perfect, thank you!  
B: You’re welcome! Enjoy your meal.

- Korean Translation:  
A: 안녕하세요, 페퍼로니 피자 라지 사이즈로 배달 주문하고 싶어요.  
B: 네! 주소와 전화번호 알려주시겠어요?  
A: 주소는 메이플 스트리트 123번지이고, 전화번호는 555-1234입니다.  
B: 알겠습니다. 토핑이나 음료 추가로 원하시는 게 있으세요?  
A: 네, 버섯도 추가해 주세요. 그리고 소다 한 병도 주세요.  
B: 알겠습니다. 총 금액은 18달러 50센트입니다. 약 30분 걸릴 거예요.  
A: 좋아요, 감사합니다!  
B: 네, 맛있게 드세요.

- Useful Expressions:  
- I’d like to order a (size) (type) pizza for delivery/pick-up.  
- Can I get extra (topping)?  
- What kinds of drinks do you have?  
- How long will it take?  
- What’s the total?  
- Do

In [40]:
# 연습 일본어 선생님 프롬프트 템플릿 설계
# 전문적인 영어 회화 튜터 프롬프트 템플릿 설계
template_japanese = """You are an experienced Japanese conversation teacher with 10 years of expertise.
Create practical Japanese conversations for the given situation with Korean translations.
Please follow the FORMAT exactly as shown below.

#SITUATION:
{question}

#FORMAT:
- Japanese Conversation:
- Korean Translation:
- Useful Expressions:
- Cultural Notes (if applicable, provide relevant cultural insights in KOREAN):
"""

# 개선된 프롬프트 템플릿 생성
prompt_japanese = PromptTemplate.from_template(template_japanese)

# ChatOpenAI 모델 객체 생성 (OpenRouter 사용)
import os

model_japanese = ChatOpenAI(
    temperature=0.5,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 문자열 출력 파서 생성
output_parser_japanese = StrOutputParser()

In [44]:
# 영어 회화 튜터 체인 구성
# 프롬프트 → 모델 → 출력 파서의 완전한 파이프라인
chain_japanese = prompt_japanese | model_japanese | output_parser_japanese


In [1]:
answer_japanese = chain_japanese.stream({"question": "도쿄에 있는 한국식 치킨집에서 주문"})
# 스트리밍 출력
stream_response(answer_japanese)

NameError: name 'chain_japanese' is not defined